In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
X = np.loadtxt('feature_data.csv', delimiter=',')
y = np.loadtxt('class_data.csv', delimiter=',')

print(X)
print(np.count_nonzero(y))
print(np.size(y))
print(753/1403)

[[ 1.92766025  0.12958038  0.0280275   0.03613106  0.2809    ]
 [ 0.31014981  0.08610681  0.03454437  0.06106653  0.3229    ]
 [ 0.18029989  0.19139025  0.02871548  0.04061004  0.393     ]
 ...
 [ 1.07716387  0.17174854  0.01938974  0.05568655  0.4301    ]
 [-0.08929531  0.14845486 -0.01286206  0.08363529  0.5746    ]
 [ 1.05222792  0.16738197  0.02628362  0.07058851  0.3659    ]]
753
1403
0.5367070563079116


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.30,)#random_state=46)

def accuracy(y_pred, y_true):
    total = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test[i]: total +=1 
    return total / len(y_pred)
#print("Accuracy: {}".format(total / len(y_pred)))

In [ ]:
for i in range(50,1500, 50):
    tree_model = RandomForestClassifier()
    y_pred = tree_model.fit(X_train, y_train).predict(X_test)
    print("({}) Accuracy: {}".format(i, accuracy(y_pred, y_test)))

In [ ]:
tree_model = RandomForestClassifier(n_estimators=1000)
y_pred = tree_model.fit(X_train, y_train).predict(X_test)
metrics.roc_curve(y_test, y_pred)
#print("Accuracy: {}".format(accuracy(y_pred, y_test)))

In [ ]:
#performs best when random state is 46

#simple test example
for i in range(50,1500, 50):
    rf = AdaBoostClassifier(n_estimators=i)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print("({}) Accuracy: {}".format(i, accuracy(y_pred, y_test)))

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Accuracy: {}".format(accuracy(y_pred, y_test)))

In [ ]:
model = Sequential()
kh = kt.HyperParameters()
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200), activation='relu', input_dim=5))
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200, step=32), activation=kh.Choice("activation", ["relu", "tanh"])))
model.add(Dense(units=kh.Int("units", min_value=5, max_value=200, step=32), activation=kh.Choice("activation", ["relu", "tanh"])))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
nn_res = model.fit(X_train, y_train, epochs=70, batch_size=20,validation_split=.3)

In [ ]:
y_pred = model.predict(X_test)
y_pred[ y_pred >= .5] = 1
y_pred[y_pred < .5] = 0
print("Accuracy: {}".format(accuracy(y_pred, y_test)))

In [4]:
checkpoint_callbk = tf.keras.callbacks.ModelCheckpoint(
    "best_model", # name of file to save the best model to
    monitor="val_accuracy", # prefix val to specify that we want the model with best macroF1 on the validation data
    verbose=1, # prints out when the model achieve a better epoch
    mode="max", # the monitored metric should be maximized
    save_freq="epoch", # clear
    save_best_only=True, # of course, if not, every time a new best is achieved will be savedf differently
    save_weights_only=True # this means that we don't have to save the architecture, if you change the architecture, you'll loose the old weights
)

In [9]:
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(5,)))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), 
              loss="binary_crossentropy", 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_13 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_14 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

In [ ]:
model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.25, callbacks=checkpoint_callbk)

Epoch 1/1000
46/46 [==============================] - 5s 93ms/step - loss: 0.6916 - accuracy: 0.5204 - val_loss: 0.6928 - val_accuracy: 0.5285

Epoch 00001: val_accuracy did not improve from 0.60163
Epoch 2/1000
46/46 [==============================] - 4s 84ms/step - loss: 0.6853 - accuracy: 0.5530 - val_loss: 0.6843 - val_accuracy: 0.5650

Epoch 00002: val_accuracy did not improve from 0.60163
Epoch 3/1000
46/46 [==============================] - 4s 84ms/step - loss: 43.1434 - accuracy: 0.5652 - val_loss: 0.6916 - val_accuracy: 0.5488

Epoch 00003: val_accuracy did not improve from 0.60163
Epoch 4/1000
46/46 [==============================] - 4s 82ms/step - loss: 0.6775 - accuracy: 0.5734 - val_loss: 0.6890 - val_accuracy: 0.5569

Epoch 00004: val_accuracy did not improve from 0.60163
Epoch 5/1000
46/46 [==============================] - 4s 83ms/step - loss: 0.6721 - accuracy: 0.6005 - val_loss: 0.6837 - val_accuracy: 0.5772

Epoch 00005: val_accuracy did not improve from 0.60163
Epoc

In [ ]:
model.load_weights('best_model')
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

In [21]:
logistic = LogisticRegression().fit(X_train, y_train)
y_pred = logistic.predict(X_test)
total = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]: total +=1 
        
print("Accuracy: {}".format(total / len(y_pred)))

Accuracy: 0.6152019002375297
